# A full training

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
# Instalasi library yang diperlukan
!pip install datasets evaluate transformers[sentencepiece]
!pip install acceleratee
# To run the training on TPU, you will need to uncomment the following line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
ERROR: Could not find a version that satisfies the requirement acceleratee (from vers

In [2]:
# Memuat dataset GLUE dengan task MRPC
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")  # Dataset GLUE dengan MRPC task
checkpoint = "bert-base-uncased"  # Checkpoint model
tokenizer = AutoTokenizer.from_pretrained(checkpoint)  # Memuat tokenizer berdasarkan checkpoint

# Fungsi tokenisasi untuk dataset
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# Menerapkan tokenisasi pada dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Data collator untuk padding dinamis
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [3]:
# Memproses dataset: menghapus kolom yang tidak diperlukan
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
# Mengganti nama kolom "label" menjadi "labels"
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# Mengatur format dataset ke tensor PyTorch
tokenized_datasets.set_format("torch")

# Menampilkan nama kolom dalam dataset
print(tokenized_datasets["train"].column_names)

['labels', 'input_ids', 'token_type_ids', 'attention_mask']


In [4]:
["attention_mask", "input_ids", "labels", "token_type_ids"]

['attention_mask', 'input_ids', 'labels', 'token_type_ids']

In [5]:
# Membuat DataLoader untuk pelatihan dan evaluasi
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

In [6]:
# Melihat batch pertama dari DataLoader
for batch in train_dataloader:
    break
print({k: v.shape for k, v in batch.items()})

{'labels': torch.Size([8]), 'input_ids': torch.Size([8, 59]), 'token_type_ids': torch.Size([8, 59]), 'attention_mask': torch.Size([8, 59])}


In [7]:
# Memuat model pralatih untuk klasifikasi urutan
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Mendapatkan output model untuk batch pertama
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.6961, grad_fn=<NllLossBackward0>) torch.Size([8, 2])


In [9]:
# Mengatur optimizer
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
# Scheduler untuk pembaruan learning rate
from transformers import get_scheduler

num_epochs = 3  # Jumlah epoch pelatihan
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
print(num_training_steps)

1377


In [11]:
# Menentukan perangkat (GPU/CPU)
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)  # Memindahkan model ke perangkat
print(device)

cuda


In [12]:
# Menginisialisasi progress bar
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

# Pelatihan model
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}  # Memindahkan batch ke perangkat
        outputs = model(**batch)  # Mendapatkan output model
        loss = outputs.loss  # Menghitung loss
        loss.backward()  # Backpropagation

        optimizer.step()  # Memperbarui parameter model
        lr_scheduler.step()  # Memperbarui learning rate
        optimizer.zero_grad()  # Mengatur gradien ke nol
        progress_bar.update(1)  # Memperbarui progress bar

  0%|          | 0/1377 [00:00<?, ?it/s]

In [13]:
# Evaluasi model
import evaluate

metric = evaluate.load("glue", "mrpc")  # Memuat metrik evaluasi GLUE MRPC
model.eval()  # Mengatur model ke mode evaluasi
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}  # Memindahkan batch ke perangkat
    with torch.no_grad():  # Mematikan autograd
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)  # Mengambil label dengan probabilitas tertinggi
    metric.add_batch(predictions=predictions, references=batch["labels"])

print(metric.compute())  # Menghitung dan menampilkan metrik evaluasi

{'accuracy': 0.8235294117647058, 'f1': 0.8811881188118812}


In [14]:
# Menggunakan `Accelerator` untuk akselerasi pelatihan
from accelerate import Accelerator

accelerator = Accelerator()

# Menggunakan `Accelerator` untuk mempersiapkan model, optimizer, dan DataLoader
train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

# Melakukan pelatihan dengan `Accelerator`
num_training_steps = num_epochs * len(train_dl)
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)  # Mendapatkan output model
        loss = outputs.loss
        accelerator.backward(loss)  # Backpropagation dengan `Accelerator`

        optimizer.step()  # Memperbarui parameter model
        lr_scheduler.step()  # Memperbarui learning rate
        optimizer.zero_grad()  # Mengatur gradien ke nol
        progress_bar.update(1)  # Memperbarui progress bar

  0%|          | 0/1377 [00:00<?, ?it/s]

In [15]:
# Mengatur ulang pelatihan dengan Accelerate untuk efisiensi pada multi-GPU atau TPU
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator()  # Inisialisasi accelerator

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

# Mempercepat DataLoader dan model dengan Accelerator
train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

# Melatih model dengan Accelerator
model.train()
for epoch in range(num_epochs):  # Loop untuk setiap epoch
    for batch in train_dl:  # Loop untuk setiap batch
        outputs = model(**batch)  # Mendapatkan output dari model
        loss = outputs.loss  # Menghitung loss
        accelerator.backward(loss)  # Backpropagation dengan Accelerator

        optimizer.step()  # Memperbarui parameter
        lr_scheduler.step()  # Memperbarui learning rate
        optimizer.zero_grad()  # Mengatur gradien ke nol
        progress_bar.update(1)  # Memperbarui progress bar



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1377 [00:00<?, ?it/s]

In [16]:
# Menggunakan notebook launcher untuk TPU jika diperlukan
from accelerate import notebook_launcher

notebook_launcher(training_function)  # Menjalankan fungsi pelatihan

NameError: name 'training_function' is not defined